In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.externals import joblib
import numpy as np
import pandas as pd
import os

TrainTestDataDir = '/home/songyue/homeCredit/HomeCreditDefaultRisk/Data/TrainTestData'
FifSaveDir = '/home/songyue/homeCredit/HomeCreditDefaultRisk/GBDT'

def loadData(DataName):
    DataPath = os.path.join(TrainTestDataDir, DataName)
    if not os.path.exists(DataPath):
        print('%s does not exist!' % DataPath)
        return
    OriginData = pd.read_csv(DataPath, index_col=0)
    OriginData = OriginData.sample(frac=1)  # 打乱顺序后返回
    return OriginData


def NormalData(TrainData, TestData):
    # 对一些列的均值大于100的进行归一化处理
    AllData = TrainData.append(TestData)
    for col in AllData.columns:
        if abs(AllData[col].mean()) > 1:
            scaler = StandardScaler().fit(np.atleast_2d(AllData[col]).T)
            TrainData[col] = scaler.transform(np.atleast_2d(TrainData[col]).T)
            TestData[col] = scaler.transform(np.atleast_2d(TestData[col]).T)
    return TrainData, TestData

def gbdt():
    gbdt_model = joblib.load(os.path.join(FifSaveDir,'gbdt2.m'))
    TrainData = pd.read_csv(os.path.join(TrainTestDataDir,'AllTrain.csv'))
    TrainData.reset_index(drop=True, inplace=True)
    TrainData.drop('SK_ID_CURR', axis=1, inplace=True)
    TestData = loadData('test.csv')
    TestID = TestData.SK_ID_CURR.values
    TestData.drop('SK_ID_CURR', axis=1, inplace=True)
    TrainData.drop('TARGET', axis=1, inplace=True)
#     _,TestData = NormalData(TrainData, TestData)
    pred = gbdt_model.predict(TestData)
    gender_submission = pd.DataFrame({'SK_ID_CURR':TestID,'TARGET':pred})
    gender_submission.to_csv(os.path.join(FifSaveDir,'result1.csv'), index = False)

if __name__ == '__main__':
    gbdt()

NotFittedError: This GradientBoostingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.